# Optimisation of analytical functions

In [ ]:
// include header files

#include <iostream>

#include "xtensor/xarray.hpp"
#include "xtensor/xnpy.hpp"
#include "xtensor/xio.hpp"

#include "xnio/ga.hpp"
#include "xnio/analytical_functions.hpp"

## Rosenbrock function

Optimise Rosenbrock function with `xnio::ga`.

Rosenbrock function is expressed as:

$$
 f(x_1, x_2) = 100(x_1^2 - x_2) + (1 - x_1)^2 \quad with \quad \bf{X} \quad \in \left[-3, 3\right]
$$

In [ ]:
std::array<std::size_t, 2> shape = {40, 2};
xt::xarray<double> X = xt::zeros<double>(shape);

xnio::Rosenbrock objective_f;

xnio::ga genetic_algorithm;
genetic_algorithm.initialise(X);

std::size_t num_generations = 300;

std::size_t no_frames = 10;
std::size_t skip = static_cast<std::size_t>(std::floor(num_generations / no_frames));

for (auto i{0}; i<num_generations; ++i)
{
    genetic_algorithm.evolve(X, objective_f, std::make_tuple(0.05),
                          std::make_tuple(), std::make_tuple(0.8),
                          std::make_tuple(0.5, 60.0));
    if (i % skip == 0)
    {
        std::string pop_file = "./output/rose_pop_" + std::to_string(i) + ".npy";
        xt::dump_npy(pop_file, X);
    }
}

std::cout << "Last pop: \n" << X << "\n" << std::endl;

## Branin function

Optimise Branin function with `xnio::ga`.

Branin function is expressed as:

$$
 f(x) = ( x_2 - \frac{5.1}{4\pi^2}x_2 + \frac{5}{\pi}x_1 - 6 )^2 + 10\left[ (1 - \frac{1}{8\pi})\cos{x_1} + 1 \right] + 5x_1\\
				with \quad x_1 \in \left[-5,10\right], x_2 \in \left[0,15\right] 
$$

In [ ]:
std::array<std::size_t, 2> shape = {40, 2};
xt::xarray<double> X = xt::zeros<double>(shape);

xnio::Branin objective_f;

xnio::ga genetic_algorithm;
genetic_algorithm.initialise(X);

std::size_t num_generations = 300;

auto y = objective_f(X);
auto sort = xt::argsort(y);
double y_best = y(sort(39));

std::size_t stall{0};

for (auto i{0}; i<num_generations; ++i)
{
    genetic_algorithm.evolve(X, objective_f, std::make_tuple(0.05),
                          std::make_tuple(), std::make_tuple(0.85),
                          std::make_tuple(0.5, 60.0));
    
    y = objective_f(X);
    sort = xt::argsort(y);
    double y_best_n = y(sort(39));
    
    if ((fabs(y_best - y_best_n) <= 1e-006))
    {
      if (stall <= 50)
      {
        ++stall;
      }
      else
      {
        std::cout << i << std::endl;
        break;
      }
    }
    else
    {
      stall = 0;
    }
        

    y_best = y_best_n;
}

std::cout << "Last pop: \n" << X << "\n" << std::endl;